<div style="text-align: center;">
    <h1>
        <a href="https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri" target="_blank">Brain Tumor Classification (MRI)</a>
    </h1>
<h3>Mohammad Nourbakhsh Marvast</h3>
</div>


# Imports

In [ ]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;
import seaborn as sns;
import sklearn;
import tensorflow as tf;

# Loading Data

In [ ]:
batch_size = 32
img_height = 244
img_width = 244

# Load the dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/brain-tumor-classification-mri/Training',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=101
)


In [ ]:
class_names = dataset.class_names

In [ ]:
for images, labels in dataset.take(1):
  plt.figure(figsize=(10, 10))
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(tf.squeeze(images[i]), cmap='gray')
    plt.title(class_names[labels[i]])
    plt.axis("off")
  plt.show()

# Model Training

In [ ]:
def normalize_image(image, label):
  image = tf.cast(image, tf.float32) / 255.0
  return image, label

ds = dataset.map(normalize_image)

In [ ]:
# Split into train and validation datasets
val_size = 0.2  # 20% for validation

# Calculate the number of batches in the validation set
val_batches = int(len(dataset) * val_size)
train_batches = len(dataset) - val_batches

# Split dataset into training and validation
train_dataset = ds.take(train_batches)
val_dataset = ds.skip(train_batches)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(img_height, img_width, 1)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation= 'relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])


model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= 0.0001),
              loss= tf.keras.losses.SparseCategoricalCrossentropy,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
stop= tf.keras.callbacks.EarlyStopping(patience= 5, restore_best_weights=True)

In [ ]:
model.fit(train_dataset,
          epochs=50,
          validation_data=val_dataset,
          callbacks= [stop]
          )

In [ ]:
pd.DataFrame(model.history.history).plot()
plt.ylabel('loss')
plt.xlabel('epochs')
plt.title('Loss vs Epochs')
plt.show()

# Model Evaluation

## Loading Testing Dataset

In [ ]:
batch_size = 32
img_height = 244
img_width = 244

# Load the dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/brain-tumor-classification-mri/Testing',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=101
)

In [ ]:
def normalize_image(image, label):
  image = tf.cast(image, tf.float32) / 255.0
  return image, label

ds_test = test_ds.map(normalize_image)

In [ ]:
model.evaluate(ds_test)

In [ ]:
y_true = []
y_pred = []

for images, labels in ds_test:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
len(y_pred), len(y_true)

In [ ]:
from sklearn import metrics

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np

print('====================')
print('Model Evaluation')
print('====================')
print()

print('====================')
print('Classification Report')
print('====================')

class_names = dataset.class_names
print(metrics.classification_report(y_true, y_pred, target_names=class_names))
print()

print('====================')
print('Confusion Matrix')
print('====================')
accuracy = metrics.accuracy_score(y_true, y_pred)
precision = metrics.precision_score(y_true, y_pred, average='weighted')
recall = metrics.recall_score(y_true, y_pred, average='weighted')

# Use unique labels from y_true for multiclass
class_labels = np.unique(y_true).tolist()  # e.g., [0, 1, 2, ...]
cm = metrics.confusion_matrix(y_true, y_pred, labels=class_labels)

disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap='PuBu', values_format='d')
ax.set_title(f'Confusion Matrix\nAccuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}')
ax.set_xticklabels(class_names, rotation=45, ha='right')
plt.show()